In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
isNotebook = '__file__' not in locals()

In [3]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *
from twinews.evaluation import metrics
from twinews.evaluation.utils import *
import time
import pymongo

In [4]:
logger = Logger(tmpDir('logs') + "/evaluation.log") if isNotebook else Logger("evaluation.log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [5]:
iterations = 1 if isNotebook else 10000000
sleep = 0 if isNotebook else 10
exceptionSleep = 10

In [6]:
metricFuncts = \
{
    'ndcg': metrics.ndcg,
    'ndcg@10': metrics.ndcgAt10,
    'ndcg@100': metrics.ndcgAt100,
    'mrr': metrics.mrr,
    'p@10': metrics.pAt10,
    'p@100': metrics.pAt100,
    'map': metrics.map,
}

In [7]:
# We get scores collection et the rankings GridFS:
twinewsScores = getTwinewsScores(logger=logger)
twinewsRankings = getTwinewsRankings(logger=logger)
# For a certain amount of iterations:
for i in range(iterations):
    # We get all
    modelsKeys = twinewsRankings.keys()
    # For all model instances:
    for modelKey in modelsKeys:
        # We init the eval data to None:
        evalData = None
        rankings = None
        # For all metrics:
        for metricKey, metricFunct in metricFuncts.items():
            # If we didn't added the score previously:
            if not twinewsScores.has({'id': modelKey, 'metric': metricKey}):
                try:
                    # We print infos:
                    log("Computing " + metricKey + " score of " + modelKey + "...", logger)
                    # We get all data:
                    meta = twinewsRankings.getMeta(modelKey)
                    splitVersion = meta['splitVersion']
                    maxUsers = meta['maxUsers']
                    modelName = meta['model']
                    # We get eval data:
                    if evalData is None:
                        log("Downloading eval data version " + str(splitVersion) + "...", logger)
                        evalData = getEvalData(splitVersion, logger=logger, maxExtraNews=0)
                    candidates = evalData['candidates']
                    # We get rankings:
                    if rankings is None:
                        log("Downloading rankings of " + modelKey + "...", logger)
                        rankings = twinewsRankings[modelKey]
                        # We check if rankings are coherent with candidates (for the right split version):
                        gotACheck = False
                        rankingsKeys = set(rankings.keys())
                        candidatesKeys = set(candidates.keys())
                        if maxUsers is None:
                        assert len(rankingsKeys) == len(candidatesKeys)
                        assert len(rankingsKeys.union(candidatesKeys)) == len(candidatesKeys)
                        for userId in rankings:
                            assert len(rankings[userId]) == len(candidates[userId])
                            for i, ranking in enumerate(rankings[userId]):
                                assert isinstance(ranking, list)
                                rankingSet = set(ranking)
                                currentCandidates = candidates[userId][i]
                                assert len(rankingSet) == len(currentCandidates)
                                assert isinstance(currentCandidates, set)
                                assert len(rankingSet.union(currentCandidates)) == len(rankingSet)
                                gotACheck = True
                        assert gotACheck
                        log("Done.", logger)
                    # We convert all in a list of rel vectors:
                    rels = []
                    for userId in rankings:
                        for ranking in rankings[userId]:
                            rel = rankingToRelevanceVector(ranking, set(evalData['testUsers'][userId].keys()))
                            rels.append(rel)
                    # We compute all scores:
                    scores = []
                    for rel in rels:
                        scores.append(metricFunct(rel))
                    # We mean all scrores:
                    score = np.mean(scores)
                    # And finally we add the score in the db:
                    addTwinewsScore(modelKey, metricKey, score, verbose=False)
                    # We print result:
                    log(metricKey + " score of " + modelKey + ": " + str(truncateFloat(score, 2)), logger)
                except Exception as e:
                    if isNotebook:
                        raise e
                    else:
                        logException(e, logger)
                        time.sleep(exceptionSleep)
    time.sleep(sleep)

twinews scores initialised.
Computing ndcg score of lda-aa8d5...
--> tic: 32.79s | message: Eval data loaded
--> toc total duration: 35.92s | message: Got Twinews evaluation data
Done.
ndcg score of lda-aa8d5: 0.25
Computing ndcg@10 score of lda-aa8d5...
ndcg@10 score of lda-aa8d5: 0.03
Computing ndcg@100 score of lda-aa8d5...
ndcg@100 score of lda-aa8d5: 0.09
Computing mrr score of lda-aa8d5...
mrr score of lda-aa8d5: 0.08
Computing p@10 score of lda-aa8d5...
p@10 score of lda-aa8d5: 0.02
Computing p@100 score of lda-aa8d5...
p@100 score of lda-aa8d5: 0.01
Computing map score of lda-aa8d5...
map score of lda-aa8d5: 0.03
Computing ndcg score of nmf-b61b9...
--> tic: 32.29s | message: Eval data loaded
--> toc total duration: 35.3s | message: Got Twinews evaluation data
Done.
ndcg score of nmf-b61b9: 0.4
Computing ndcg@10 score of nmf-b61b9...
ndcg@10 score of nmf-b61b9: 0.18
Computing ndcg@100 score of nmf-b61b9...
ndcg@100 score of nmf-b61b9: 0.31
Computing mrr score of nmf-b61b9...
mr

AssertionError: 